# Итоговое задание по Проекту 5. Выбираем авто выгодно

### В этом соревновании предстоит построить модель, которая будет прогнозировать стоимость автомобиля по его характеристикам. 
### Задание выполнил Бегунов Павел (DST-56)

<img src="https://whatcar.vn/media/2018/09/car-lot-940x470.jpg"/>

# Импорт библиотек

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import re

from datetime import datetime

from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, RobustScaler
from sklearn.feature_selection import f_classif, mutual_info_classif
from sklearn.decomposition import PCA

from tqdm.notebook import tqdm

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.neighbors import KNeighborsRegressor
from catboost import CatBoostRegressor
import xgboost as xgb

from sklearn.ensemble import GradientBoostingRegressor

from sklearn.base import clone

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
pd.set_option('display.max_columns', 50)

In [ ]:
# Зафиксируем RANDOM_SEED, чтобы эксперименты были воспроизводимы:
RANDOM_SEED = 42
# Зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt
# Укажем текущую версию ноутбука:
VERSION = 3

 # Предобработка

In [ ]:
def rename_columns(df):
    new_column_names = df.rename(columns={'bodyType': 'body_type',
                                          'engineDisplacement': 'engine_displacement',
                                          'enginePower': 'engine_power',
                                          'fuelType': 'fuel_type',
                                          'modelDate': 'model_date',
                                          'model': 'model_name',
                                          'numberOfDoors': 'number_of_doors',
                                          'priceCurrency': 'price_currency',
                                          'productionDate': 'production_date',
                                          'vehicleConfiguration': 'vehicle_configuration',
                                          'vehicleTransmission': 'vehicle_transmission',
                                          'Комплектация': 'complectation_dict',
                                          'Владельцы': 'owners',
                                          'Владение': 'ownership_time',
                                          'ПТС': 'vehicle_passport',
                                          'Привод': 'drive_unit',
                                          'Руль': 'rudder_side',
                                          'Состояние': 'status',
                                          'Таможня': 'customs'
                                          }, inplace=True)
    return new_column_names


def reduce_mem_usage(df, verbose=True):
    '''
    Function allows to reduce memory usage. 
    '''
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage(deep=True).sum() / 1024**2
    for column in df.columns:
        col_type = df[column].dtypes
        if col_type in numerics:
            c_min = df[column].min()
            c_max = df[column].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[column] = df[column].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[column] = df[column].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[column] = df[column].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[column] = df[column].astype(np.int64)
            else:
                c_prec = df[column].apply(lambda x: np.finfo(x).precision).max()
                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max and c_prec == np.finfo(np.float32).precision:
                    df[column] = df[column].astype(np.float32)
                else:
                    df[column] = df[column].astype(np.float64)
    end_mem = df.memory_usage(deep=True).sum() / 1024**2
    if verbose: 
        print(f'Memory usage decreased to {end_mem:5.2f} Mb ({100 * (start_mem - end_mem) / start_mem:.1f}% reduction)')
    return df.sample(5)


def missing_data(df):
    '''
    The function displays the number and percentage of skips for each column.
    '''
    for column in df.columns:
        missing = df[column].isnull().sum()
        percent_of_missing = np.mean(df[column].isnull())
        if missing > 0:
            print(f'{column} - {missing} value(s), {percent_of_missing*100:.2f}%')


def duplicate_data(df):
    '''
    The function displays the number of duplicates for each column.
    '''
    if len(df) > len(df.drop_duplicates()):
        display(df[df.duplicated()].head(3))
        print(f'{df[df.duplicated()].shape[0]} duplicate(s) were found')
    else:
        print(f'Duplicates not found')


def IQR_outlier(df, column, mode='analysis', verbose=True):
    '''
    Displays and returns the boundaries of the interquartile range.
    
    Parameters
    ----------
    df : DataFrame
        Two-dimensional, size-mutable, potentially heterogeneous tabular data.
    
    column : column : index
        Column labels to use for dataframe.
    
    mode: string, optional (default='analysis')
        Interquartile range analysis mode.
        * If 'analysis',then displays the name of the feature, interquartile range,
        the boundaries of outliers, their amount and what percentage of outliers in the sample.
        * If 'correction', then returns the low and high bound for the outliers. 
    
    verbose: bool, optional (default=True)
        Controls the verbosity of the interquartile range analysis. 
    '''
    perc25 = np.percentile(df[column], 25, axis=0)
    perc75 = np.percentile(df[column], 75, axis=0)
    IQR = perc75 - perc25
    low = perc25 - 1.5*IQR
    high = perc75 + 1.5*IQR
    anomaly = len(df[df[column] > high]) + \
        len(df[df[column] < low])
    if verbose:
        if mode == 'analysis':
            print(f'Наименование признака: {column}')
            print('25-й перцентиль: {},'.format(perc25)[:-1], '75-й перцентиль: {},'.format(perc75),
            'IQR: {}, '.format(IQR), 'Границы выбросов: [{f}, {l}].'.format(f=low, l=high))
            print(f'Выбросов, согласно IQR: {anomaly} | {anomaly/len(df):2.2%}')
        elif mode == 'correction':
            return low, high


def get_log(df, column):
    '''
    Function for logarithm of a feature.
    '''
    return df[column].apply(lambda x: np.log(x + 1))
        
        
def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred - y_true) / y_true))

# Импорт данных

In [ ]:
DIR_TRAIN = '../input/parsing-all-moscow-auto-ru-09-09-2020/' 
DIR_TEST = '../input/sf-dst-car-price-prediction/'

In [ ]:
!ls '../input'
# тренировочный датасет(train, используется для обучения модели)
train = pd.read_csv(DIR_TRAIN + 'all_auto_ru_09_09_2020.csv') 
rename_columns(train)
# тестовый датасет(test, используется для оценки точности модели)
test = pd.read_csv(DIR_TEST + 'test.csv')
rename_columns(test)
sample_submission = pd.read_csv(DIR_TEST + 'sample_submission.csv')

In [ ]:
display(train.head())
display(test.head())
display(sample_submission.head())

# Обзор данных
### Полученный тренировочный датасет состоит из двадцати шести столбцов, содержащих следующую информацию:
* body_type: тип корпуса автомобиля;
* brand: марка автомобиля;
* color: цвет автомобиля;
* complectation_dict: комплектация;
* customs: таможня;
* description: описание;
* drive_unit: привод;
* engine_displacement: объем двигателя;
* engine_power: мощность двигателя;
* fuel_type: тип топлива;
* hidden: скрытая информация;
* mileage: пробег автомобиля;
* model_name: название модели;
* model_date: дата начала производства модели;
* name: название в каталоге;
* number_of_doors: количество дверей; 
* owners: количество владельцев;
* ownership_time: время владения данной моделью автомобиля;
* price: цена автомобиля;
* production_date: дата производства автомобиля;
* rudder_side: параметр, указывающий на какой стороне автомобиля находится руль;
* start_date: дата добавления объявления;
* status: состояние автомобиля; 
* vehicle_сonfiguration: конфигурация автомобиля;
* vehicle_passport: паспорт транспортного средства;
* vehicle_transmission: трансмиссия.

### Признаки из тестового датасета, которых нет в тренировочном:
* car_url: ссылка на объявление;
* equipment_dict: оборудование;
* image: фотография автомобиля;
* model_info: информация о модели;
* parsing_unixtime: время получения данных;
* price_currency: в какой валюте указана цена;
* super_gen: сводка;
* vendor: продавец.

In [ ]:
print(f'Размер датасета для обучения: {train.shape}')
print(f'Размер датасета для тестирования: {test.shape}')

In [ ]:
# Проверим данные на наличие пропусков:
missing_data(train)
print('====' * 10)
missing_data(test)

In [ ]:
# Проверим данные на наличие дубликатов:
duplicate_data(train)
print('====' * 10)
duplicate_data(test)

In [ ]:
# Посмотрим из каких признаков состоят датасеты:
print(f'Number of train features = {len(train.columns)}:\n{sorted(train.columns)}')
print()
print(f'Number of test features = {len(test.columns)}:\n{sorted(test.columns)}')

In [ ]:
# Посмотрим какие типы признаков у нас могут быть:
train.nunique(dropna=False)

In [ ]:
test.nunique(dropna=False)

#### Резюме: 
* изначально в тренировочном наборе данных представлено **26** признаков, а в тестовом - **32**;
* некоторые признаки, такие как: **"hidden", "start_date"**, **"price"**, есть только в тренировочных данных. Признаки: **"car_url"**, **"equipment_dict"**, **"image"**, **"model_info"**, **"parsing_unixtime"**, **"price_currency"**, **"sell_id"**, **"vendor"**, **"super_gen"** присутствуют только в тестовых данных. Детально рассмотрим данные, после чего можно будет принять решение что делать с признаками;
* в обоих датасетах присутствуют пропуски. Признаки **"status"** и **"hidden"** в тренировочном наборе данных имеют **100%** пропущенных значений и подлежат удалению, так же как и **"ownership_time"**, который имеет **67.33%** пропусков. В тестовом наборе данных признаки **"complectation_dict"**, **"ownership_time"** имеют **81.50%** и **65.42%** пропущенных значений, соответственно, и подлежат удалению. Действия над другими признаками, имеющими пропуски, будут осуществляться при их детальном рассмотрении;
* в тренировочных данных есть дуликаты от которых придется избавиться;
* признак **"price"** является целевой переменной. Из обзора видно, что в данном признаке есть **410 (0,46%)** пропущенных значений, от которых нужно избавиться.  
* В качестве оценки качества модели используется **MAPE** (средний абсолютный процент ошибки), таким образом в результате работы необходимо получить наименьшее значение **MAPE**.

# Предварительная обработка данных
## Обработка пропущенных и дублированных значений

In [ ]:
# Рассмотрим признак complectation_dict, который в тестовом наборе данных имеет 81.50% пропусков:
print(f'Значений вида {train["complectation_dict"].unique()[0]}: \
{train["complectation_dict"].value_counts(normalize=True)[0]:.2%}')

In [ ]:
# Рассмотрим признак status, который в тренировочном наборе наборе данных имеет 100% пропусков:
test['status'].value_counts(normalize=True)

* Признак **"complectation_dict"** имеет **66,16%** значений, которые по сути является пропусками;
* Признак **"status"** имеет только одно значение **"Не требует ремонта"**, поэтому является не информативным.

In [ ]:
# Удалим признаки, где количество пропущенных значений больше 50%:
train.drop(columns=['status', 'hidden', 'ownership_time', 'complectation_dict'], axis=1, inplace=True)
test.drop(columns=['status', 'ownership_time', 'complectation_dict'], axis=1, inplace=True)

In [ ]:
# Рассмотрим признак body_type на предмет пропусков:
train[train['body_type'].isna() == True]

In [ ]:
train.drop(train.index[[24624]], inplace=True)

* Видно, что данное объявление практически не заполнено, поэтому было принято решение его удалить.

In [ ]:
# Рассмотрим признак owners, который имеет 13.81% пропусков в тренировочном наборе данных:
train['owners'].value_counts(dropna=False)

In [ ]:
# Проверим на признаке mileage предположение о том, что NaN в owners означает, что автомобиль новый:
train[train['mileage'] == 0].shape[0]

In [ ]:
train['vehicle_condition'] = pd.isna(train['owners']).astype('int8')  # сохраним в тренировочных данных информацию о состоянии автомобилей
test['vehicle_condition'] = pd.isna(test['owners']).astype('int8')  # сохраним в тестовых данных информацию о состоянии автомобилей
train['owners'].fillna(0, inplace=True)  # заполним пропуски константой

* Пропуски в признаке **"owners"** количественно соответствуют **0** значениям признака **"mileage"**. Что подтверждает предположение о том, что данные автомобили являются новыми. На данном этапе пропуски были заполнены константой. И сохранена информацию о текущем соотоянии автомобиля - признак **"vehicle_condition"**, где 1 означает новый автомобиль, 0 - бывший в употреблении).

In [ ]:
# Рассмотрим признак description, который имеет 3.7% пропусков в тренировочном наборе данных:
train['description'].head(5)

In [ ]:
train.drop('description', axis=1, inplace=True)
test.drop('description', axis=1, inplace=True)

* Признак **"description"** - это описания текущих владельцев автомобилей. На данном этапе данный признак был удален.

In [ ]:
# Рассмотрим признак vehicle_passport, который имеет 0.41% пропусков в тренировочном наборе данных и 1 пропуск в тестовом:
train['vehicle_passport'].value_counts(dropna=False)

In [ ]:
test['vehicle_passport'].value_counts(dropna=False)

In [ ]:
# Сохраним информацию о пропусках в признаке vehicle_passport:
train['absent_documents'] = pd.isna(train['vehicle_passport']).astype('int8')
test['absent_documents'] = pd.isna(test['vehicle_passport']).astype('int8')

In [ ]:
# Заполним пропуски наиболее часто встречающимися значениями:
train_mode = train['vehicle_passport'].mode()
train['vehicle_passport'].fillna(train_mode[0], inplace=True)
test_mode = test['vehicle_passport'].mode()
test['vehicle_passport'].fillna(test_mode[0], inplace=True)

* Пропуски в данных о наличии паспорта транспортного средства были заполнены наиболее часто встречающимися значениями. Однако информация о пропусках была сохранена, так как есть вероятность, что документов на автомобиль попросту нет.

In [ ]:
# Рассмотрим признак equipment_dict, который имеет 28.82% пропусков в тестовом набое данных:
test['equipment_dict'][0]

In [ ]:
test.drop('equipment_dict', axis=1, inplace=True)

* Признак **"equipment_dict"** был удален ввиду его отсутствия в тренировочном датасете.

In [ ]:
# Удалим пропуски в целевой переменной:
train.dropna(subset=['price'], inplace=True)

In [ ]:
# Уберём дубликаты из тренировочного набора данных:
train.drop_duplicates(inplace=True)

#### Резюме:
* После обработки пропусков и дубликатов тренировочный набор данных уменьшился на **3,65% (3,262)** и составляет **86,116** значений. Тестовый - остался без изменений.
* Количество признаков в тренировочном и тестовом наборе данных уменьшилось на **15.4%** и ровняется **22** и **28**, соответственно.

## Обработка признаков и стандартизация данных

In [ ]:
# Рассмотрим признак start_date из тренировочного набора данных:
pd.to_datetime(train['start_date']).dt.year

In [ ]:
# Рассмотрим признак parsing_unixtime из тeстового набора данных:
test['parsing_unixtime'] = pd.to_datetime(test['parsing_unixtime'], unit='s')

In [ ]:
# Посмотрим в какой период были получены тестовые данные:
print(f"{test['parsing_unixtime'].dt.year.unique()}\n{test['parsing_unixtime'].dt.month_name().unique()}\n{sorted(test['parsing_unixtime'].dt.day.unique())}")

In [ ]:
# Определим в какой день было получено больше всего объявлений:
test['parsing_unixtime'].dt.day.value_counts()

In [ ]:
# Сохраним в переменные даты получения данных:
test_parsing_time = pd.to_datetime('21OCT2020')  # допущение, связанное с публикацией большинства объявлений в этот день
train_parsing_time = pd.to_datetime('09SEP2020')

In [ ]:
display(train.sample(3))
display(test.sample(3))

#### Приведем данные к единой форме:

In [ ]:
# body_type:
train['body_type'] = train['body_type'].str.lower().str.split(' ').apply(lambda x: x[0])
test['body_type'] = test['body_type'].str.lower().str.split(' ').apply(lambda x: x[0])

# brand:
train['brand'] = train['brand'].str.lower()
test['brand'] = test['brand'].str.lower()

# color:
color_dict = {'040001': 'чёрный', 'EE1D19': 'красный', '0000CC': 'синий', 'CACECB': 'серебристый',
              '007F00': 'зелёный', 'FAFBFB': 'белый', '97948F': 'серый', '22A0F8': 'голубой',
              '660099': 'пурпурный', '200204': 'коричневый', 'C49648': 'бежевый', 'DEA522': 'золотистый', 
              '4A2197': 'фиолетовый', 'FFD600': 'жёлтый', 'FF8649': 'оранжевый', 'FFC0CB': 'розовый'}
train['color'] = train['color'].map(color_dict)

# model_date:
train['model_date'] = train['model_date'].astype(np.uint16)
test['model_date'] = test['model_date'].astype(np.uint16)

# number_of_doors:
train['number_of_doors'] = train['number_of_doors'].astype(np.uint8)
test['number_of_doors'] = test['number_of_doors'].astype(np.uint8)

# vehicle_transmission:
vehicle_transmission_dict = {'MECHANICAL': 'механическая', 'AUTOMATIC': 'автоматическая', 'VARIATOR': 'вариатор', 'ROBOT': 'роботизированная'}
train['vehicle_transmission'] = train['vehicle_transmission'].map(vehicle_transmission_dict)

# engine_power:
train['engine_power'] = train['engine_power'].astype(np.uint16)
test['engine_power'] = test['engine_power'].str.split(' ').apply(lambda x: x[0]).astype(np.int16)

# rudder_side:
train['rudder_side'] = train['rudder_side'].apply(lambda x: 'Левый' if x == 'LEFT' else 'Правый')

# vehicle_passport:
train['vehicle_passport'] = train['vehicle_passport'].apply(lambda x: 'Оригинал' if x == 'ORIGINAL' else 'Дубликат')

# owners:
train['owners'] = train['owners'].astype(np.uint8)
test['owners'] = test['owners'].apply(lambda x: x[:1]).astype(np.uint8)

# model_name:
train['model_name'] = train['model_name'].str.lower()
test['model_name'] = test['model_name'].str.lower()

# price:
train['price'] = train['price'].astype(np.int64)

In [ ]:
# Признак engine_displacement:
pattern = re.compile('\d\.\d')
train_engine_displacement = train['name'].apply(lambda x: re.findall(pattern, x))
train['engine_displacement'] = train_engine_displacement.apply(lambda x: -1 if not x or x[0] == '0.0' else float(x[0]))

test_engine_displacement = test['engine_displacement'].apply(lambda x: re.findall(pattern, x))
test['engine_displacement'] = test_engine_displacement.apply(lambda x: -1 if not x else float(x[0]))

In [ ]:
# Удалим признаки, которые являются не информативными или отсутствуют в одном из наборов данных:
train.drop(columns=['start_date', 'vehicle_configuration', 'customs', 'name'], axis=1, inplace=True)   # 'model_name'

test.drop(columns=['car_url', 'image', 'model_info', 'parsing_unixtime',
                   'price_currency', 'sell_id', 'super_gen', 'vendor',
                   'vehicle_configuration', 'customs', 'name'], axis=1, inplace=True)  # 'model_name'

* Есть очень старые объявления, хотя явной причины, объясняющей почему эти позиции не были реализованы, в данных не обнаружено. Не стал использовать признак **"start_date"** в качестве фильтра;
* **"car_url"** и **"image"**: ссылки на объявления и фото автомобилей, не пригодятся при построении модели;
* **"model_info"**: практически повторяет признак model_name. Является не информативным; 
* Все цены указаны в российских рублях, поэтому **"price_currency"** не информативен;
* **"sell_id"**: индекс объявления, не пригодится для построения модели;
* **"super_gen"** полезный признак, в котором указаны все технические параметры автомобиля, однако многие параметры уже выделены в отдельных признаках;
* **"vendor"**: признак, указывающий поставщика, отсутствует в тренировочных данных; 
* **"parsing_unixtime"**: период получения тестовых данных, с небольшим допущением, был сохранен в переменную;
* **"vehicle_configuration"** не информативный признак;
* **"customs"**: не информативный признак, так как все автомобили растоможены.

In [ ]:
display(train.sample(3))
display(test.sample(3))

In [ ]:
# Для дальнейшей работы над признаками объединяем тренировочные и тестовые данные в один датасет
train['sample'] = 1  # помечаем где тренировочные данные
test['sample'] = 0  # помечаем где тестовые данные
test['price'] = 0  # в тестовых данных нет значения цены автомобиля, мы её должны предсказать, поэтому пока просто заполняем нулями
df = test.append(train, sort=False).reset_index(drop=True)  # объединение

In [ ]:
# Оптимизируем затраты используемой памяти:
reduce_mem_usage(df)

### Резюме:
* В ходе обработки данных было сгенерировано два новых признака **"vehicle_condition"**, **"absent_documents"**, которые учитывают состояние автомобилей и наличие документов к ним.
* После обработки данных и объединения тренировочного и тестового датасетов получилось:
    * Размер объединенного датасета составляет **120802** наблюдений;
    * Количестов признаков - **19**. Признак **"sample"** не учитывается, так как искусствено создан для удобства разделения обучающей и тестовой выборки.

## Числовые признаки

In [ ]:
# Исходя из обзора данных создадим список признаков, значения которых похожи на числовые:
num_columns = ['engine_power', 'mileage', 'model_date', 'production_date']
df[num_columns + ['price']].sample(5)

In [ ]:
# Проведем анализ для выявления выбросов числовых признаков:
for column in num_columns:
    IQR_outlier(df.query('sample == 1'), column)
    print('=='*20)

In [ ]:
# Построим boxplots для числовых признаков:
fig, axes = plt.subplots(2, 2, figsize=(18,15))
for i, column in enumerate(num_columns):
    sns.boxplot(x=df.query('sample == 1')[column],
                hue=df.query('sample == 1')['price'],
                ax=axes.flat[i],
                showmeans=True,
                meanline=True,
                orient='h');

In [ ]:
# Посмотрим на распределения числовых признаков:
fig, axes = plt.subplots(2, 2, figsize=(18,15))
for i,col in enumerate(num_columns):
    sns.distplot(df.query('sample ==1')[col], ax=axes.flat[i], bins=100, kde=True)

In [ ]:
# Посмотрим на распределение значений целевого признака:
fig = plt.subplots(figsize=(10, 6))
sns.distplot(df.query('sample == 1')['price'].apply(lambda x: x * 0.001), bins=50, kde=True);

In [ ]:
# Посмотрим статистику по числовым признакам:
df.query('sample == 1')[num_columns + ['price']].describe()

### Резюме:
* Признак **"engine_power"**. Минимальное значение - **11**, максимальное - **800**. Среднее значение - **170.6**, медианное - **150**. Имеет тяжелый правый хвост. Есть выбросы в количестве **5165**. Будет проводиться логарифмирование данного признака. Нашел информацию об автомобилях с большой мощностью двигателя, поэтому выбросы пока не будут удалены.
* Признак **"mileage"**. Распределение признака похоже на нормальное, но имеется пик в районе нуля, как выяснилось это связано с наличием объявлений о продаже новых автомобилей. Есть выбросы в количестве **1414**. Выбросы не вижу смысла удалять, поскольку ответить на вопрос как эксплуатировались автомобили не представляется возможным.
* Признаки **"model_date"** и **"production_date"**. Распределения данных признаков смещены влево, присутствуют выбросы в количестве **2314** и **1836** соответственно. Так как в продаже могут быть раритетные автомобили, то выбросы устраняться не будут.   
* Распределение целевого признака (**"price"**) имеет тяжелый правый хвост. Поэтому есть смысл взять логарифм данного признака.

In [ ]:
# Создадим переменную model_age, которая отражает возраст автомобиля:
# model_age = df['production_date'].apply(lambda x: 2020 - x)  # год, когда были получены данные
# model_age = model_age.apply(lambda x: x if x != 0 else x + 1)  # добавим 1, чтобы избежать деления на 0 

In [ ]:
# # Создадим признак mileage_per_year - годовой пробег:
# df['mileage_per_year'] = df['mileage'] / model_age
# num_columns.append('mileage_per_year')

In [ ]:
# Построим матрицу корреляций для числовых признаков:
f = plt.subplots(figsize=(10, 6))
sns.heatmap(df.query('sample == 1')[num_columns].corr().abs(), vmin=0,
            vmax=1, annot=True, fmt=".2f", linewidths=0.1);

In [ ]:
# Определим значимость числовых признаков с помощью функции mutual_info_classif:
fig = plt.subplots(figsize=(7, 5))
imp_num = pd.Series(f_classif(df.query('sample == 1')[num_columns], df.query('sample == 1')['price'])[0], index = num_columns)
imp_num.sort_values(inplace = True)
imp_num.plot(kind = 'barh');

### Резюме:
* Наиболее значимым для целевой переменной является признак **"production_date"**.
* Признаки **"model_date"** и **"production_date"** сильно скоррелированы. Решением может быть удаление менее значимого признака или применение метода декомпозиции. 
* Также признак **"mileage"** имеет сильную корреляцию с признаками **"model_date"** и **"production_date"**.
* **На практике, добавление новых признаков, устранение выбросов и борьба с мультиколлинеарностью приводит к ухудшению результатов, поэтому принято решение не вносить каких-либо изменений в числовые признаки**.

## Категориальные признаки

In [ ]:
cat_columns = ['body_type', 'brand', 'model_name', 'color', 'engine_displacement',
               'fuel_type', 'number_of_doors', 'vehicle_transmission',
               'drive_unit', 'owners']
df[cat_columns].sample(5)

### model_name

In [ ]:
# Создадим новый признак, отражающий количество моделей авто, входящих в линейку бренда:
lineup_dict = dict(df.groupby('brand')['model_name'].nunique())
df['number_of_models'] = df['brand'].map(lineup_dict)

In [ ]:
# Обновим списки признаков:
cat_columns.remove('model_name')
cat_columns.append('number_of_models')

In [ ]:
# Удалим признак model_name из набора данных:
df.drop('model_name', axis=1, inplace=True)

### color

In [ ]:
sns.countplot(y = df.query('sample == 1')['color'], order = df.query('sample == 1')['color'].value_counts().index);

Разделим цвета по степени распространенности от наиболее распространенных(**1**) до редких(**4**).

In [ ]:
# Модифицируем признак color по степени распространенности:
common_color_dict = {'чёрный': 1, 'красный': 4, 'синий': 3, 'серебристый': 3,
                     'зелёный': 4, 'белый': 2, 'серый': 3, 'голубой': 4,
                     'пурпурный': 4, 'коричневый': 4, 'бежевый': 4, 'золотистый': 4, 
                     'фиолетовый': 4, 'жёлтый': 4, 'оранжевый': 4, 'розовый': 4}
df['color'] = df['color'].map(common_color_dict)

### engine_displacement

In [ ]:
# Классифицируем автомобили с учетом объема двигателя:
df['engine_displacement_classification'] = df['engine_displacement'].apply(lambda x: 'electro' if x == -1 else\
                                                                                     'minicar' if x < 1.2 else\
                                                                                     'subcompact' if x < 1.8 else\
                                                                                     'average_size' if x < 3.6 else 'large_capacity')
cat_columns.append('engine_displacement_classification')

In [ ]:
# Посмотрим на распределения категориальных признаков:
fig, axes = plt.subplots(6, 2, figsize=(15,20))
for i,col in enumerate(cat_columns):
    sns.countplot(df.query('sample == 1')[col], ax=axes.flat[i-1]);

In [ ]:
# Проведем Label Encoding категориальных признаков:
le = LabelEncoder()
for column in cat_columns:
    df[column] = le.fit_transform(df[column])
    print(dict(enumerate(le.classes_)))
    
df[cat_columns].sample(5)

In [ ]:
# Построим матрицу корреляций для категориальных признаков:
f = plt.subplots(figsize=(10, 6))
sns.heatmap(df.query('sample == 1')[cat_columns].corr().abs(), vmin=0,
            vmax=1, annot=True, fmt=".2f", linewidths=0.1);

In [ ]:
# Определим значимость категориальных признаков с помощью функции mutual_info_classif:
fig = plt.subplots(figsize=(7, 5))
imp_cat = pd.Series(mutual_info_classif(df.query('sample == 1')[cat_columns], df.query('sample == 1')['price'], discrete_features=True), index=cat_columns)
imp_cat.sort_values(inplace=True)
imp_cat.plot(kind='barh');

#### Резюме:
* Были добавлены новые признаки **"number_of_models"**, **"engine_displacement_classification"**. Был изменен признак **"color"**, теперь он отражает не конкретный цвет автомобиля, а в какой диапазон он входит по степени распространенности цветов.
* Максимальное значение коэффициента корреляции составляет **0,57** между признаками **"drive_unit"** и **"body_type"**. Это свидетельствует о некоторой корреляции между признаками. Оценка значимости категориальных признаков показывает, что все признаки достаточно сильно влияют на целевой. Самым значимым из  признаков является **"brand"**.

## Бинарные признаки

In [ ]:
# Исходя из обзора данных создадим список бинарных признаков:
bin_columns = ['rudder_side', 'vehicle_passport', 'vehicle_condition', 'absent_documents']
df[bin_columns].sample(5)

In [ ]:
# Проведем Label Encoding бинарных признаков:
le = LabelEncoder()
for column in bin_columns:
    df[column] = le.fit_transform(df[column])
    print(dict(enumerate(le.classes_)))
    
df[bin_columns].sample(5)

In [ ]:
# Построим матрицу корреляций для бинарных признаков:
f = plt.subplots(figsize=(10, 6))
sns.heatmap(df[bin_columns].corr().abs(), vmin=0,
            vmax=1, annot=True, fmt=".2f", linewidths=0.1);

In [ ]:
# Определим значимость бинарных признаков с помощью функции mutual_info_classif:
fig = plt.subplots(figsize=(7, 5))
imp_bin = pd.Series(mutual_info_classif(df.query('sample == 1')[bin_columns], df.query('sample == 1')['price'],
                                        discrete_features=True), index=bin_columns)
imp_bin.sort_values(inplace=True)
imp_bin.plot(kind='barh');

### Резюме:
* Максимальное значение коэффициента корреляции составляет **0,17** между признаками **vehicle_condition** и **absent_documents**. Это говорит о достаточно слабой корреляции между признаками.
* Оценка значимости бинарных признаков показывает, что наиболее значимым из признаков является **"vehicle_condition"**, который указывает какой автомобиль выставлен на продажу: новый или подержанный.

# Подготовка данных к обучению

In [ ]:
# Разделим объединенный датасет на тренировочный и тестовый:
train = df.query('sample == 1').drop(['sample'], axis=1)
sub = df.query('sample == 0').drop(['sample', 'price'], axis=1)

In [ ]:
# Создадим массив числовых признаков и стандартизируем его при помощи StandardScaler:
standart_scaler = StandardScaler()
X_train_num = standart_scaler.fit_transform(train[num_columns])
X_sub_num = standart_scaler.fit_transform(sub[num_columns])
print(X_train_num.shape, X_sub_num.shape)

In [ ]:
# Создадим список категориальных признаков для OneHotEncoding:
cat_ohe_columns = ['body_type', 'brand', 'color','fuel_type', 'vehicle_transmission', 'drive_unit']

# Создадим массив категориальных признаков и сделаем OneHotEncoding:
X_train_cat = train[cat_columns].drop(cat_ohe_columns, axis=1).values
X_sub_cat = sub[cat_columns].drop(cat_ohe_columns, axis=1).values
print(X_train_cat.shape, X_sub_cat.shape)

# OneHotEncoding:
ohe = OneHotEncoder(sparse=False).fit(df[cat_ohe_columns])
X_train_ohe_cat = ohe.transform(train[cat_ohe_columns])
X_sub_ohe_cat = ohe.transform(sub[cat_ohe_columns])
print(X_train_ohe_cat.shape, X_sub_ohe_cat.shape)

# Объединим преобразованные признаки:
X_train_cat = np.hstack([X_train_cat, X_train_ohe_cat])
X_sub_cat = np.hstack([X_sub_cat, X_sub_ohe_cat])
print(X_train_cat.shape, X_sub_cat.shape)

In [ ]:
# Создадим массив бинарных признаков:
X_train_bin = train[bin_columns].values
X_sub_bin = sub[bin_columns].values
print(X_train_bin.shape, X_sub_bin.shape)

In [ ]:
# Объединяем в одно признаковое пространство тренировочную выборку: 
X = np.hstack([X_train_num, X_train_cat,  X_train_bin])
y = train['price'].values
print(X.shape, y.shape)

In [ ]:
# Объединяем в одно признаковое пространство тестовую выборку:
X_sub = np.hstack([X_sub_num, X_sub_cat,  X_sub_bin])
X_sub.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=RANDOM_SEED)

## Model 1: LinearRegression

In [ ]:
model = LinearRegression(n_jobs=-1).fit(X_train, np.log(y_train + 1))
predict = np.exp(model.predict(X_test)) 
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict))*100:0.2f}%")

## Model 2 : CatBoost

In [ ]:
model = CatBoostRegressor(iterations = 5000,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE'],
                          silent=True,
                         )
model.fit(X_train, np.log(y_train),
         #cat_features=cat_features_ids,
         eval_set=(X_test, np.log(y_test)),
         verbose_eval=0,
         use_best_model=True,
         #plot=True
         )

model.save_model('catboost_single_model_2_baseline.model')

predict_test = np.exp(model.predict(X_test))

In [ ]:
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_test))*100:0.2f}%")

## Model 3: XGBoost

### Гипермараметры

In [ ]:
max_depth = [8, 12, 14, 16]
n_estimators = [100, 500, 1000]
hyperparameters = dict(max_depth=max_depth, n_estimators=n_estimators)
model = xgb.XGBRegressor()
model.fit(X_train, y_train)

clf = GridSearchCV(model, hyperparameters, cv=3, scoring='neg_mean_absolute_error')

best_model = clf.fit(X_train, y_train).best_estimator_

In [ ]:
# Посмотрим на лучшие гиперпараметры:
best_parameters = best_model.get_params()
for param_name in sorted(best_parameters.keys()):
        print('\t%s: %r' % (param_name, best_parameters[param_name]))

### Обучение модели

In [ ]:
xb = xgb.XGBRegressor(objective='reg:squarederror', colsample_bytree=0.5, learning_rate=0.03, \
                      max_depth=14, alpha=1, n_jobs=-1, n_estimators=1000)
xb.fit(X_train, np.log(y_train + 1))
print(f"Точность модели по метрике MAPE: {(mape(y_test, np.exp(xb.predict(X_test))))*100:0.2f}%")
predict_submission = np.exp(xb.predict(X_sub))

# max_depth = 8, MAPE = 12.85%
# max_depth = 12, MAPE = 12.35%
# max_depth = 12, MAPE = 12.32%
# max_depth = 16, MAPE = 12.34%

## Gradient Boosting

In [ ]:
gb = GradientBoostingRegressor(min_samples_split=2, learning_rate=0.03, max_depth=5, n_estimators=1000)
gb.fit(X_train, np.log(y_train+1))
print(f"Точность модели по метрике MAPE: {(mape(y_test, np.exp(gb.predict(X_test))))*100:0.2f}%")

## Stacking

In [ ]:
scaler = StandardScaler() 
X_train = scaler.fit_transform(X_train) 
X_test = scaler.transform(X_test) 


y_train = y_train 
y_test = y_test

cv = KFold(n_splits=5, shuffle=True, random_state=RANDOM_SEED)


def compute_meta_feature(model, X_train, X_test, y_train, cv):
    X_meta_train = np.zeros_like(y_train, dtype=np.float32)  
    
    for train_fold_index, predict_fold_index in cv.split(X_train):
        X_fold_train, X_fold_predict = X_train[train_fold_index], X_train[predict_fold_index]
        y_fold_train = y_train[train_fold_index]

        folded_model = clone(model)
        folded_model.fit(X_fold_train, y_fold_train)

        X_meta_train[predict_fold_index] = folded_model.predict(X_fold_predict)

    meta_model = clone(model)
    meta_model.fit(X_train, y_train)

    X_meta_test = meta_model.predict(X_test)

    return X_meta_train, X_meta_test


def generate_meta_features(model, X_train, X_test, y_train, cv):
    features = [compute_meta_feature(model, X_train, X_test, y_train, cv) for model in tqdm(model)]    
    stacked_features_train = np.vstack([features_train for features_train, features_test in features]).T
    stacked_features_test = np.vstack([features_test for features_train, features_test in features]).T
    return stacked_features_train, stacked_features_test

X_train = np.where(np.isnan(X_train), 0, X_train)
X_test = np.where(np.isnan(X_test), 0, X_test)
y_train = np.where(np.isnan(y_train), 0, y_train)

In [ ]:
stacked_features_train, stacked_features_test = generate_meta_features([
                            RandomForestRegressor(n_estimators=300, min_samples_split=2, min_samples_leaf=1, 
                                                  max_features=3, max_depth=19, bootstrap=True, random_state=RANDOM_SEED),
                            GradientBoostingRegressor(min_samples_split=2, learning_rate=0.03, max_depth=10, n_estimators=300),
                            KNeighborsRegressor(n_neighbors=2, algorithm = 'ball_tree', weights = 'distance', p=1),
                            RandomForestRegressor(random_state = RANDOM_SEED, n_jobs = -1, verbose = 1, max_depth=5, n_estimators=200),
                            ExtraTreesRegressor(random_state=RANDOM_SEED), 
                            RandomForestRegressor(random_state=RANDOM_SEED, max_depth=15)], X_train, X_test, y_train, cv)

In [ ]:
def compute_metric(model, X_train, y_train, X_test, y_test): 
    model.fit(X_train, y_train) 
    y_test_pred = model.predict(X_test) 
    return np.round(mape(y_test, y_test_pred)*100, 2)
print(f"Точность модели по метрике MAPE: {compute_metric(model, stacked_features_train, y_train, stacked_features_test, y_test)}%")

## Резюме: 
* Исследования показали, что наилучшего результата удается достичь применив модель **XGBoost**. 
* При дальнейшей работе с данными расчёты будут вестить с помощью **XGBoost**, **CatBoost** и **LinearRegression**, с целью снижения затрачиваемого на исследования времени.
* Закомментировал некоторые модели, потому как они затрачивают много времени на расчёт. 

# История работы над проектом
* **VERSION 0**: была сделана первичная обработка и исследование данных, добавлен признак **vehicle_condidtion**. Модель **CatBoost**, метрика **MAPE** = **12.77%** 
* **VERSION 1**: добавлены различные модели. Контрольные метрики:
    * **CatBoost**: **MAPE** = **12.77%**
    * **XGBoost**: **MAPE** = **12.37%**
* **VERSION 2**: 
    * Были добавлены новые категориальные признаки **"number_of_models"**, **"engine_displacement_classification"**. 
    * Признак **"color"** изменен по степени распространенности цветов в объявлениях. 
    * Установлено, что любые изменения, а также добавление новых признаков в числовые переменные приводят к ухудшению результатов исследования.
    * Оптимизирована обработка(Label encoding, One Hot encoding) категориальных признаков.
    * Стандартизация не изменила результаты исследования. Применялись StandardScaler и RobustScaler из библиотеки scikit learn.
    * Осуществлен подбор гиперпараметров модели XGBoost. 
    * Контрольные метрики:
        * **CatBoost**: **MAPE** = **12.78%**
        * **XGBoost**: **MAPE** = **12.32%** 

# Submission

In [ ]:
sample_submission['price'] = predict_submission
sample_submission.to_csv(f'submission_v{VERSION}.csv', index=False)
sample_submission.head(10)